# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2022-02-07	Improving Methods for Multi-Terrain Classification Beyond Visual Perception	2021 Fifth IEEE International Conference on Robotic Computing (IRC)	Terrain classification in mixed-surface unstructured environments is key for safe navigation, energy efficiency, and anticipating motion volatility. This is particularly true for dynamically moving legged platforms which are highly impacted by foot ground interactions. This research demonstrates terrain classification using a long short-term memory (LSTM) model trained on actuator time series data, particularly the difference in center-of-pressure (COP) and leg forces. The LSTM COPForce model showed a 97.5% accuracy in classification on three outdoor surfaces with small amounts of data and no additional sensors.	"C. Allred, M. Russell, M. Harper and J. Pusey, ""Improving Methods for Multi-Terrain Classification Beyond Visual Perception,"" 2021 Fifth IEEE International Conferenc

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2022-02-07,Improving Methods for Multi-Terrain Classifica...,2021 Fifth IEEE International Conference on Ro...,Terrain classification in mixed-surface unstru...,"C. Allred, M. Russell, M. Harper and J. Pusey,...",Improving-Terrain-Classification,https://ieeexplore.ieee.org/abstract/document/...
1,2022-09-26,Divide and Survey: Observability Through Multi...,2022 IEEE International Smart Cities Conferenc...,Deploying autonomous drone systems in smart ci...,"H. Kocabas, C. Allred and M. Harper, ""Divide a...",Multi-Drone-City-Roadway-Coverage,https://ieeexplore.ieee.org/abstract/document/...
2,2023-01-24,Terrain Dependent Power Estimation for Legged ...,2022 Sixth IEEE International Conference on Ro...,Gait-based legged robots offer substantial adv...,"C. Allred, H. Kocabas, M. Harper and J. Pusey,...",Terrain-Power-Estimation-Legged-Robots,https://ieeexplore.ieee.org/document/10023912
3,2023-11-1,Unknown building exploration simulator (UBES),Elsevier Journal Software Impacts,The Unknown Building Exploration Simulator (UB...,"Allred, Christopher, Huzeyfe Kocabas, and Mari...",Unknown-Building-Exploration,https://www.sciencedirect.com/science/article/...
4,2023-12-11,Detecting Ballistic Motions in Quadruped Robot...,2023 Seventh IEEE International Conference on ...,Quadrupedal robots require sophisticated algor...,"C. Allred, J. Pusey and M. Harper, ""Detecting ...",Detecting-Ballistic-Motions-in-Quadruped-Robot...,https://ieeexplore.ieee.org/abstract/document/...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [11]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"

    # no_cite_gen
    md += "\ncite_generation: false"
    
    md += "\n---"
    
    # Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\n### Recommended citation: \n\n" + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [10]:
!ls ../_publications/

2022-02-07-Improving-Terrain-Classification.md
2022-09-26-Multi-Drone-City-Roadway-Coverage.md
2023-01-24-Terrain-Power-Estimation-Legged-Robots.md
2023-11-1-Unknown-Building-Exploration.md
2023-12-11-Detecting-Ballistic-Motions-in-Quadruped-Robots-BTMC-for-RL.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
